# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --db uk_food --collection establishments --file "C:\Users\kappa\Documents\Bootcamp\UofU-VIRT-DATA-PT-02-2024-U-LOLC\02-Homework\12-NoSQL-Databases\Starter_Code\Resources\establishments.json" --jsonArray`

In [7]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [8]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [9]:
# confirm that our new database was created
databases = mongo.list_database_names()
pprint(databases)

['admin', 'classDB', 'config', 'local', 'met', 'my_database', 'uk_food']


In [10]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [11]:
# review the collections in our new database
collections = db.list_collection_names()

# Print the list of collections
print("Collections in the 'uk_food' database:")
for collection in collections:
    print(collection)

Collections in the 'uk_food' database:
establishments


In [12]:
# review a document in the establishments collection
collection = db['establishments']

document = collection.find_one()

# Print the document using pretty-printing
print("Document in the 'establishments' collection:")
pprint(document)


Document in the 'establishments' collection:
{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66a034e807cca6d6d90c6149'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource':

In [13]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [21]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "name": "Penang Flavours",
    "location": "Greenwich",
    "cuisine": "Halal",
    "rating": None 
}

# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant)

# Retrieve the newly inserted document using its unique properties
inserted_document = establishments.find_one({"_id": result.inserted_id})

# Print the name of the newly inserted document
print("Inserted document name:", inserted_document["name"])


Inserted document name: Penang Flavours


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [24]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

result = collection.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},  # Query to match BusinessType
    {"_id": 0, "BusinessTypeID": 1, "BusinessType": 1}  # Projection to return only BusinessTypeID and BusinessType
)

print("BusinessTypeID and BusinessType:", result)


BusinessTypeID and BusinessType: {'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [26]:
# Update the new restaurant with the correct BusinessTypeID

business_type_id = "your_business_type_id_here"
result = establishments.update_one(
    {"name": "Penang Flavours"},  # Query to find the specific restaurant
    {"$set": {"BusinessTypeID": business_type_id}}  # Update operation
)

# Print the result of the update operation
if result.matched_count > 0:
    print("Updated document:", result.modified_count, "document(s) updated.")
else:
    print("No document found with the name 'Penang Flavours'.")


Updated document: 1 document(s) updated.


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [27]:
# Find how many documents have LocalAuthorityName as "Dover"
count = establishments.count_documents({"LocalAuthorityName": "Dover"})

# Print the count
print("Number of documents with LocalAuthorityName 'Dover':", count)

Number of documents with LocalAuthorityName 'Dover': 994


In [28]:
# Delete all documents where LocalAuthorityName is "Dover"
result = establishments.delete_many({"LocalAuthorityName": "Dover"})

# Print the result
print("Number of documents deleted:", result.deleted_count)

Number of documents deleted: 994


In [29]:
# Check if any remaining documents include Dover
remaining_document = establishments.find_one({"LocalAuthorityName": "Dover"})

# Print the result
if remaining_document:
    print("There are remaining documents with LocalAuthorityName 'Dover'.")
else:
    print("No remaining documents with LocalAuthorityName 'Dover'.")

No remaining documents with LocalAuthorityName 'Dover'.


In [30]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one()

# Print the result
if remaining_document:
    print("There are remaining documents in the collection.")
else:
    print("No remaining documents in the collection.")

There are remaining documents in the collection.


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [31]:
# Change the data type from String to Decimal for longitude and latitude
result = establishments.update_many(
    {},
    [{
        "$set": {
            "latitude": {"$toDouble": "$latitude"},
            "longitude": {"$toDouble": "$longitude"}
        }
    }]
)

# Print the result
print(f"Matched {result.matched_count} documents.")
print(f"Modified {result.modified_count} documents.")

Matched 38789 documents.
Modified 38789 documents.


Use `update_many` to convert `RatingValue` to integer numbers.

In [32]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [33]:
# Change the data type from String to Integer for RatingValue
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many(
    {"RatingValue": {"$in": non_ratings}},
    {"$set": {"RatingValue": None}}
)

result = establishments.update_many(
    {"RatingValue": {"$type": "string"}},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]  
)

print(f"Matched {result.matched_count} documents.")
print(f"Modified {result.modified_count} documents.")

Matched 34694 documents.
Modified 34694 documents.


In [34]:
# Check that the coordinates and rating value are now numbers
latitude_check = establishments.find_one({"latitude": {"$type": "double"}})
longitude_check = establishments.find_one({"longitude": {"$type": "double"}})
rating_value_check = establishments.find_one({"RatingValue": {"$type": "int"}})

# Print results
if latitude_check:
    print("There are documents with latitude as a double.")
else:
    print("No documents with latitude as a double.")

if longitude_check:
    print("There are documents with longitude as a double.")
else:
    print("No documents with longitude as a double.")

if rating_value_check:
    print("There are documents with RatingValue as an integer.")
else:
    print("No documents with RatingValue as an integer.")

No documents with latitude as a double.
No documents with longitude as a double.
There are documents with RatingValue as an integer.
